Plotting the ERA5 and CARRA data to compare the climate means

In [3]:
import metview as mv

In [1]:
#retrieve the data from mv
import copy


def request_vars(params,dt,type='an',step=0,origin='no-ar-ce',database=None):
    ds = {}
    vars = copy.deepcopy(params)
    paramlist = [vars[param]['param'] for param in vars]
    ret = mv.retrieve(type=type,
                      levtype='sfc',
                      param=paramlist,
                      date=dt.strftime("%Y-%m-%d"),
                      expver='prod',
                      origin=origin,
                      class_='rr',
                      time=dt.strftime("%H"),
                      database=database,
                      stream='oper',
                      step=step)
    #
    x = ret.to_dataset()
    attrs = x.variables[list(vars.keys())[0]].attrs
    #
    lon0 = attrs['GRIB_LoVInDegrees']
    lat0 = attrs['GRIB_LaDInDegrees']
    lat1 = attrs['GRIB_Latin1InDegrees']
    lat2 = attrs['GRIB_Latin2InDegrees']
    nx = attrs['GRIB_Nx']
    ny = attrs['GRIB_Ny']
    #   
    lons = x.longitude.values
    lats = x.latitude.values
    #
    lons2 = np.where(lons>180,lons-360,lons)
    lon0 = np.where(lon0>180,lon0-360,lon0)
    #
    proj = ccrs.LambertConformal(central_latitude=lat0,
                                 central_longitude=lon0,
                                 standard_parallels=(lat1, lat2))
    ds['misc'] = {'date':dt,
                  'lons':lons2,
                  'lats':lats,
                  'proj':proj,
                  'fcstep':step}
    #
    for param in vars:
        missVal = x.variables[param].attrs['GRIB_missingValue']
        val = ma.masked_values(x.variables[param].values,missVal)
        vars[param]['field'] = val
    #
    ds['params'] = vars
    return ds



In [4]:
fin_era5 = "era5-t2m-20210129.grib2"
era5 = mv.read(fin_era5)
fin_carra="t2m-mean_carra-20210101-20210131.grib2"
carra = mv.read(fin_carra)



In [25]:
carra_ds = carra.to_dataset()
type(carra_ds)
#carra_ds.variables.keys()
#attrs = carra_ds.variables["time"].attrs #["2 metre temperature"].attrs
#attrs.keys()


Ignoring index file '/home/cap/data/scripts/dataviz/metviz/python/carra/t2m-mean_carra-20210101-20210131.grib2.923a8.idx' incompatible with GRIB file


xarray.core.dataset.Dataset

In [32]:
carra_ds.data_vars
carra_ds.coords

Coordinates:
    time               datetime64[ns] ...
    step               timedelta64[ns] ...
    heightAboveGround  float64 ...
    latitude           (y, x) float64 ...
    longitude          (y, x) float64 ...
    valid_time         datetime64[ns] ...

In [3]:
mv.grib_get(era5, ['shortName', 'dataDate', 'dataTime',
                 'stepRange', 'validityDate', 'validityTime'])

[['2t', '20210101', '0000', '0', '20210101', '0']]

In [4]:
mv.grib_get(carra, ['shortName', 'dataDate', 'dataTime',
                 'stepRange', 'validityDate', 'validityTime'])

[['2t', '20210101', '0000', '0', '20210101', '0']]

In [5]:
print(mv.minvalue(era5), mv.maxvalue(era5))


223.11875915527344 307.94493103027344


In [6]:
#north = mv.latitudes(era5).max()
north = mv.latitudes(carra).max() + 2
south = mv.latitudes(carra).min() - 2
east = mv.longitudes(carra).max() + 2
west = mv.longitudes(carra).min() - 2
[north, south, east, west]

[88.05481975728765, 53.809957665030055, 361.999315108157, -1.9999170364162637]

max_points = mv.gfind(wg0, max0, max0*0.05)


north = mv.latitudes(max_points).max() + 2
south = mv.latitudes(max_points).min() - 2
east = mv.longitudes(max_points).max() + 2
west = mv.longitudes(max_points).min() - 2
[north, south, east, west]

In [7]:
view = mv.geoview(
    map_area_definition = "corners",
    area                = [south, west, north, east]
    )

In [8]:
mv.plot(view, carra) #, coloured_markers)


Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

Python - ERROR  - 20230607.123438 - Line 0 in 'PythonScript': uPlotBatch run failed with exit code: 35584


Exception: Metview error: Error calling Metview function